<a href="https://colab.research.google.com/github/k1151msarandega/1st-order/blob/main/QC_Module_2_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1eBlhnvWo94RnzPK-86F5MzfFd86Cjb9C" width="65">


Created by: The AVELA Team
# **0. Welcome to the <a style="text-decoration:none;" href="https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093"><font color='blue'>G</font><font color='red'>o</font><font color='Goldenrod'>o</font><font color='blue'>g</font><font color='green'>l</font><font color='red'>e</font><font color="black"> Colab</font></a> notebook!**
Make sure to read <u>every</u> cell in this notebook to get your Quantum Computing [Python](https://docs.google.com/document/d/1gen8uhv7UC_Qo5wT5paX8tesrSffhXlK9WDYunmcWgs/edit?usp=sharing) code working! For more guidance, you can check out the [python documentation](https://docs.python.org/3/tutorial/index.html) to learn more python and the qiskit documentation](https://docs.quantum.ibm.com/). If you do not know how to code in at all here is a curated [AVELA Python Basics Course](https://tinyurl.com/AVELA-python-basics) to start with. \\
To run code, all you have to do is click the *run* button ▶️ (triangle in a circle). \\


Your job will be to read every block and **replace** the question marks (?) in each coding cell with the **correct** information explained in the comments. Then run the cell! \\

NOTE: If there are no question marks (?) in a cell, then just click the *run* button!

###[Session 1 Module 1 for reference](https://colab.research.google.com/drive/1MEU4kwcO8Vjxk5cSX4wQy1J092hqgUlp#scrollTo=ZoBqYbepYbdK)

###[Completion form](https://forms.gle/gRkpWVayCo5wScyBA)

In [ ]:
#@title ↓Install qiskit Library by Clicking "run" { display-mode: "form" }

!pip install qiskit qiskit-aer qiskit-ibm-runtime matplotlib numpy

## All Qiskit Import Modules

In [ ]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import QuantumCircuit
from qiskit.primitives import BackendSamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

#🧪 Qiskit Runtime Setup Tutorial

This tutorial walks you through setting up the Qiskit Runtime environment, getting your IBM Quantum API key, and running a quantum job.

### 🔑 Step 1: Get Your IBM Quantum API Token

1. Go to [https://quantum.ibm.com](https://quantum.ibm.com)
2. Log in or create an account.
3. Click your **profile icon** (top-right) → **Account settings**
4. Under **API Token**, click **Copy token**

### Step 2: Initialize Qiskit Runtime Service

Replace `'Your_API_Key'` with your actual API token obtained in Step 1.


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your account details (do this only once per runtime session)
service = QiskitRuntimeService(
    token='Your_API_Key',  # Replace with your actual token
    channel='ibm_quantum',
    instance='ibm-q/open/main'  # You can update this if using a different hub/group/project
)

print(service.active_account())  # Check if account is loaded
print(service.backends())        # List available backends

### Step 3: Select a Quantum Backend

Choose a real device to run on. You can use the least busy backend or specify one like `"ibm_brisbane"`

In [ ]:
# Option 1: Automatically select least busy real device (uncomment if desired)
# backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)

# Option 2: Use a specific backend
backend = service.backend("ibm_brisbane")

print(backend.name)


### Step 4: Set Up a Simulator with Noise Model (Optional)

This uses the characteristics of the real backend in a simulator for faster testing.

In [ ]:
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

noise_model = NoiseModel.from_backend(backend)
backend_sim = AerSimulator(noise_model=noise_model)

### Step 5: Initialize the Samplers

These are used to run quantum circuits.

In [ ]:
from qiskit.primitives import BackendSamplerV2
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Simulator-based sampler
sampler_sim = BackendSamplerV2(backend=backend_sim)

# Runtime-based sampler for real devices
sampler_runtime = Sampler(backend=backend)


You're now ready to build circuits and run them on both simulators and real quantum devices using Qiskit Runtime! 🎉

# Quantum Key Distribution (QKD) with an Eavesdropper

We will implement exactly the same protocol as before. This time, we will insert another set of measurements, by Eve, between Alice and Bob.

## 🧭 What You Should Already Have Done

Before starting this task, make sure you've completed:

- ✅ **Section 1**: Introduction to qubits and measurement  
- ✅ **Section 2**: Basis choice and BB84 protocol fundamentals  
- ✅ **Section 3**: Circuit implementation for Alice and Bob   

---

## Goal of This Task

We are going to simulate a **QKD scenario with an eavesdropper (Eve)**.  
The protocol will be the same as before, **except that Eve measures each qubit in a random basis before it reaches Bob**.

This simulates a **man-in-the-middle attack**, and lets us observe how eavesdropping affects the error rate and key generation.

---

## What You Will Do

In this section, you'll:

1. **Re-use** the BB84 setup from Section 3 (Alice sends qubits to Bob).  
2. Insert an **Eve block** that:
   - Measures each qubit in a random basis (X or Z).
   - Re-encodes the qubit and passes it to Bob.  
3. Analyze how Eve's intervention increases the **quantum bit error rate (QBER)**.

---

## What’s Happening Technically?

| Step | Without Eve                | With Eve                        |
|------|----------------------------|---------------------------------|
| 1    | Alice prepares qubits      | Alice prepares qubits          |
| 2    | Qubits go directly to Bob  | Eve intercepts and measures     |
| 3    |                            | Eve resends qubits to Bob       |
| 4    | Bob measures qubits        | Bob measures tampered qubits    |

Eve’s interference **alters some qubits**, which increases the error rate when Alice and Bob compare their results.

---

## Implementation Tips

- Reuse your `QuantumCircuit` code from earlier tasks.
- Use a **third QuantumRegister** for **Eve’s qubits**.
- For each qubit:
  - Eve randomly selects a measurement basis (Z or X).
  - She measures the qubit and **re-prepares** it in the corresponding state.
- After Eve, Bob continues as usual: chooses a basis and performs a measurement.

---

## Analysis

After you run the simulation:

- Compare Alice and Bob’s final keys.
- Calculate the **Quantum Bit Error Rate (QBER)**:

QBER = (Number of mismatches) / (Total bits after basis reconciliation)


- Discuss how Eve’s presence increased the error rate compared to the original protocol.

---

## Let’s Get Started

Once you're ready, start by building your Qiskit circuit with **three registers**: `alice`, `eve`, and `bob`, and follow the protocol steps described above.

You can now continue coding this part in your next code cell.  
Happy hacking!


## Qiskit Setup and Alice’s State Preparation

In [ ]:
# Step 1: Set up quantum registers and generate Alice's random bits and bases
import ? as ?

bit_num = ?  # Set the number of qubits (e.g. 20)

# TODO: Initialize quantum and classical registers
qr = QuantumRegister(?, 'q')  # Quantum register
cr = ClassicalRegister(?, 'c')  # Classical register
qc = QuantumCircuit(qr, cr)

# TODO: Create a random number generator and generate Alice's random bits and bases
rng = ?.random.?(?)  # Fill in correct module/method
abits = np.round(?.random(?, ?))  # Bits (0 or 1)
abase = np.round(?.random(?, ?))  # Bases (0 = Z, 1 = X)

# Print to verify
print("Alice's bits: ", abits)
print("Alice's bases:", abase)


## Alice’s Quantum State Encoding

In [ ]:
# Step 2: Encode Alice's quantum states based on bit and basis

# TODO: Apply appropriate gates based on Alice’s bit and basis
for n in range(bit_num):
    if abits[n] == 0 and abase[n] == 1:
        qc.?  # Apply H to encode |+⟩
    elif abits[n] == 1 and abase[n] == 0:
        qc.?  # Apply X to encode |1⟩
    elif abits[n] == 1 and abase[n] == 1:
        qc.?  # Apply X
        qc.?  # Then H to encode |-⟩

qc.?  # Add a barrier after Alice’s preparation


## Eve's Measurement Setup

In [ ]:
# Step 3: Eve measures the qubits in a random basis

# TODO: Generate Eve’s random measurement bases (0=Z, 1=X)
ebase = np.round(rng.?(?, ?))  # Use appropriate RNG

# TODO: For each qubit, if Eve uses X basis, apply H before measurement
for m in range(bit_num):
    if ebase[m] == 1:
        qc.?(?)  # Apply H to rotate to X basis
    qc.?(?, ?)  # Measure qubit m into classical bit m


## Transpilation and Execution (Eve’s Measurement)

In [ ]:
# Step 4: Transpile and simulate Eve’s measurement

# TODO: Provide a backend with target information
target = ?.target
pm = generate_preset_pass_manager(target=target, optimization_level=?)

# TODO: Run the transpiler pass manager
qc_isa = ?.run(?)


## Extracting Eve’s Measured Bits

In [ ]:
# Step 5: Run the circuit and extract Eve's measurement outcome

job = ?.run([qc_isa], shots=1)
counts = job.result()[0].data.c.get_counts()

# Extract the single bitstring and reverse the order
key = list(?.keys())[0]
emeas = list(?)  # Convert string to list of characters

# Convert to integer list and reverse bit order
emeas_ints = []
for n in range(bit_num):
    emeas_ints.append(int(?[n]))
ebits = emeas_ints[::-1]

print("Eve's measured bits:", ebits)


## Eve Re-Encodes States to Send to Bob

In [ ]:
# Step 6: Eve prepares new qubits based on her measurement to send to Bob

# TODO: Create new registers and quantum circuit
qr = QuantumRegister(?, 'q')
cr = ClassicalRegister(?, 'c')
qc = QuantumCircuit(qr, cr)

# TODO: Encode Eve's guessed states using ebits and ebase
for n in range(bit_num):
    if ebits[n] == 0 and ebase[n] == 1:
        qc.?(?)  # Prepare |+⟩
    elif ebits[n] == 1 and ebase[n] == 0:
        qc.?(?)  # Prepare |1⟩
    elif ebits[n] == 1 and ebase[n] == 1:
        qc.?(?)
        qc.?(?)

qc.?  # Add a barrier


## Bob’s Measurement and Post-Processing

In [ ]:
# Step 7: Bob generates random measurement bases and measures qubits

bbase = np.round(rng.?(?, ?))

for m in range(bit_num):
    if bbase[m] == 1:
        qc.?  # Apply H to measure in X basis
    qc.?(?, ?)  # Measure qubit m into classical bit m

# Transpile and simulate
qc_isa = pm.run(qc)
job = sampler_sim.run([qc_isa], shots=1)
counts = job.result()[0].data.c.get_counts()

# Extract Bob's measured bits
key = list(counts.keys())[0]
bmeas = list(key)
bmeas_ints = [int(b) for b in bmeas]
bbits = bmeas_ints[::-1]
print("Bob's bits: ", bbits)


## Comparing Alice and Bob’s Bits

In [ ]:
# Step 8: Compare Alice and Bob’s bits where their bases match

agoodbits = []
bgoodbits = []
match_count = 0

for n in range(bit_num):
    if ?[n] == ?[n]:  # Matching basis
        agoodbits.append(int(abits[n]))
        bgoodbits.append(bbits[n])
        if int(abits[n]) == bbits[n]:
            match_count += 1

# Output results
print("Matched Alice bits: ", agoodbits)
print("Matched Bob bits:   ", bgoodbits)
print("Fidelity = ", match_count / len(agoodbits))
print("Loss = ", 1 - match_count / len(agoodbits))


## Compare QKD with and without eavesdropping on a real quantum computer

#### Just Click Run On the Code Base Below and Review the Output

In [ ]:
# This calculation was run on ibm_sherbrooke on 11-7-24 and required 3 s to run, with 127 qubits.
# Qiskit Patterns Step 1: Mapping your problem to a quantum circuit

bit_num = 127
qc = QuantumCircuit(bit_num,bit_num)

# QKD Step 1: Generate Alice's random bits and bases

abits = np.round(rng.random(bit_num))
abase = np.round(rng.random(bit_num))

# Alice's state preparation

for n in range(bit_num):
    if abits[n] == 0:
        if abase[n] == 1:
            qc.h(n)
    if abits[n] == 1:
        if abase[n] == 0:
            qc.x(n)
        if abase[n] == 1:
            qc.x(n)
            qc.h(n)

# QKD Step 2: Random bases for Bob

bbase = np.round(rng.random(bit_num))

for m in range(bit_num):
    if bbase[m] == 1:
        qc.h(m)
    qc.measure(m,m)


# Qiskit Patterns Step 2: Transpilation

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
qc_isa = pm.run(qc)

# Load the Runtime primitive and session
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode = backend)

# Qiskit Patterns Step 3: Execute

job = sampler.run([qc_isa],shots = 1)
counts=job.result()[0].data.c.get_counts()
countsint=job.result()[0].data.c.get_int_counts()

# Qiskit Patterns Step 4: Post-processing
# Extract Bob's bits

keys=counts.keys()
key=list(keys)[0]
bmeas=list(key)
bmeas_ints=[]
for n in range(bit_num):
    bmeas_ints.append(int(bmeas[n]))
bbits = bmeas_ints[::-1]

# Compare Alice's and Bob's measurement bases and collect usable bits

agoodbits=[]
bgoodbits=[]
match_count=0
for n in range(bit_num):
    if abase[n]==bbase[n]:
        agoodbits.append(int(abits[n]))
        bgoodbits.append(bbits[n])
        if int(abits[n])==bbits[n]:
            match_count+=1

# Print some results

print("Alice's bits = ", agoodbits)
print("Bob's bits = ", bgoodbits)
print("fidelity = ",match_count/len(agoodbits))
print("loss = ",1-match_count/len(agoodbits))

#### With no eavesdropping, we obtained almost 100% fidelity over this set of 127 trial bits, resulting in 55 matched bases and usable key bits. Now let's repeat this experiment with Eve listening in:

In [ ]:
# This calculation was run on ibm_nazca on 11-7-24 and required 2 s to run, with 127 qubits.
# Qiskit Patterns Step 1: Mapping your problem to a quantum circuit

bit_num = 127
qr = QuantumRegister(bit_num, 'q')
cr = ClassicalRegister(bit_num, 'c')
qc = QuantumCircuit(qr, cr)

# QKD Step 1: Generate Alice's random bits and bases

abits = np.round(rng.random(bit_num))
abase = np.round(rng.random(bit_num))

# Alice's state preparation

for n in range(bit_num):
    if abits[n] == 0:
        if abase[n] == 1:
            qc.h(n)
    if abits[n] == 1:
        if abase[n] == 0:
            qc.x(n)
        if abase[n] == 1:
            qc.x(n)
            qc.h(n)


# Eavesdropping happens here!
# Generate Eve's random measurement bases

ebase = np.round(rng.random(bit_num))

for m in range(bit_num):
    if ebase[m] == 1:
        qc.h(m)
    qc.measure(qr[m],cr[m])

# Qiskit Patterns Step 2: Transpile

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
qc_isa = pm.run(qc)

from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode = backend)

# Qiskit Patterns Step 3: Execute

job = sampler.run([qc_isa],shots = 1)
counts=job.result()[0].data.c.get_counts()
countsint=job.result()[0].data.c.get_int_counts()

# Qiskit Patterns Step 4: Post-processing
# Extract Eve's bits

keys=counts.keys()
key=list(keys)[0]
emeas=list(key)
emeas_ints=[]
for n in range(bit_num):
    emeas_ints.append(int(emeas[n]))
ebits = emeas_ints[::-1]

#print(ebits)

# Restart process
# Qiskit Patterns Step 1: Mapping your problem to a quantum circuit

# QKD Step 1: Eve uses her measurements above to prepare best guess states to send on to Bob

qr = QuantumRegister(bit_num, 'q')
cr = ClassicalRegister(bit_num, 'c')
qc = QuantumCircuit(qr, cr)


# Eve's state preparation

for n in range(bit_num):
    if ebits[n] == 0:
        if ebase[n] == 1:
            qc.h(n)
    if ebits[n] == 1:
        if ebase[n] == 0:
            qc.x(n)
        if ebase[n] == 1:
            qc.x(n)
            qc.h(n)

# QKD Step 2: Random bases for Bob

bbase = np.round(rng.random(bit_num))

for m in range(bit_num):
    if bbase[m] == 1:
        qc.h(m)
    qc.measure(qr[m],cr[m])

# Qiskit Patterns Step 2: Transpile

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
qc_isa = pm.run(qc)

# Qiskit Patterns Step 3: Execute

job = sampler.run([qc_isa],shots = 1)
counts=job.result()[0].data.c.get_counts()
countsint=job.result()[0].data.c.get_int_counts()

# Qiskit Patterns Step 4: Post-processing
# Extract Bob's bits

keys=counts.keys()
key=list(keys)[0]
bmeas=list(key)
bmeas_ints=[]
for n in range(bit_num):
    bmeas_ints.append(int(bmeas[n]))
bbits = bmeas_ints[::-1]

# Compare Alice's and Bob's bases, when they are the same, keep the bits.

agoodbits=[]
bgoodbits=[]
match_count=0
for n in range(bit_num):
    if abase[n]==bbase[n]:
        agoodbits.append(int(abits[n]))
        bgoodbits.append(bbits[n])
        if int(abits[n])==bbits[n]:
            match_count+=1

# Print some results

print("Alice's bits = ", agoodbits)
print("Bob's bits = ", bgoodbits)
print("fidelity = ",match_count/len(agoodbits))
print("loss = ",1-match_count/len(agoodbits))

# Quantum Teleportation (Communication)

## 1. Introduction and Background to Quantum Teleportation

Quantum teleportation is a technique in quantum physics that allows the **transfer of quantum information** from one place to another *without moving particles*. Unlike sci-fi teleportation, **no physical matter is transported**. Instead, it uses a fascinating quantum property: **entanglement**.

Through a clever combination of:
- **Quantum entanglement**
- **Measurement**
- **Classical communication**

…we can transfer the *state* of one qubit to another distant qubit. In this module, we’ll **break down the math** and then **implement quantum teleportation on a real IBM quantum computer**. 🚀

📚 For extra background, check out:
- [Quantum Information Basics by John Watrous](https://learning.quantum.ibm.com/course/basics-of-quantum-information)
- [Teleportation Section](https://learning.quantum.ibm.com/course/basics-of-quantum-information/entanglement-in-action#teleportation)

---

### 💡 Classical vs Quantum Bits

- **Classical bit** → can be `0` or `1`
- **Quantum bit (qubit)** → can be in a state:  
  $$|\psi\rangle = \alpha_0|0\rangle + \alpha_1|1\rangle$$  
  where $\alpha_0, \alpha_1 \in \mathbb{C}$ and  
  $$|\alpha_0|^2 + |\alpha_1|^2 = 1$$

**Measurement collapses** this state into one of the basis states:

- Probability of measuring `0`:  
  $$P_0 = |\alpha_0|^2$$

- Probability of measuring `1`:  
  $$P_1 = |\alpha_1|^2$$

---

### 🔗 What is Entanglement?

Qubits can be **entangled**, meaning measuring one affects the other *instantly*, no matter the distance.

Let Alice and Bob each have a qubit in the state $|0\rangle$:

$$
|00\rangle = |0\rangle_A \otimes |0\rangle_B
$$

This is called **little-endian notation**  
(Qiskit uses this: qubit 0 appears on the **right**).

We can create an **entangled superposition** like:

$$
\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)
$$


In this state:
- If **Bob measures `0`**, Alice **must** get `0`
- If **Bob measures `1`**, Alice **must** get `1`

Even though **they didn’t start as 0 or 1**, the measurement reveals the same result on both ends.

---

### 🌍 Entanglement Over Distance

Entanglement doesn't vanish with space. If Alice and Bob **move apart** after entangling, their qubits remain connected. Each entangled pair (like above) is called an **e-bit**.

These e-bits are **resources** for quantum communication.

> **❓ Faster than light?**  
> Unfortunately, **no**. Teleportation uses **classical communication** alongside entanglement.  
> The "spooky action at a distance" doesn't violate relativity—information isn't transferred faster than light. [See this Nature article](https://www.nature.com/articles/nature15759)

---

### 👥 Alice, Bob, and Qubit Q

Imagine:
- Alice and Bob start together, entangle a pair of qubits.
- They **separate**, taking one entangled qubit each.
- Alice receives a **mystery qubit Q** (with unknown state).
- Her task: **send the state of Q to Bob**, *without measuring Q directly*.

How? Quantum teleportation. 🧙‍♀️✨

---

### 🔧 What You’ll Need: Quantum Gates

To make this possible, we’ll use key **quantum gates** and **measurements**, and we'll:
1. Set up the entangled state
2. Use Alice's measurements
3. Send classical info to Bob
4. Have Bob apply corrections to **recreate Q's state on his qubit**

> Let’s dive into the math and code implementation next! ⚙️🧠💻


## 2. Quantum Operators (Gates)

*Feel free to skip this section if you're already familiar with quantum gates.*  
If you'd like a refresher, check out the [Basics of Quantum Information](https://learning.quantum.ibm.com/course/basics-of-quantum-information), especially the first two lessons on the IBM Quantum Learning platform.

In this teleportation protocol, we primarily use:

- **Hadamard (H) gate**
- **CNOT gate**

Other gates used (less frequently):

- **X gate**
- **Z gate**
- **SWAP gate**

---

### Matrix/Vector Representation

The states we've introduced so far are represented as vectors:

$$
|0\rangle = \begin{pmatrix}1 \\ 0\end{pmatrix} \quad , \quad
|1\rangle = \begin{pmatrix}0 \\ 1\end{pmatrix}
$$

So an arbitrary qubit state:

$$
|\psi\rangle = a|0\rangle + b|1\rangle \quad \Rightarrow \quad
|\psi\rangle = \begin{pmatrix}a \\ b\end{pmatrix}
$$

---

For two-qubit systems:

$$
|00\rangle = \begin{pmatrix}1 \\ 0 \\ 0 \\ 0\end{pmatrix}, \quad
|01\rangle = \begin{pmatrix}0 \\ 1 \\ 0 \\ 0\end{pmatrix}, \quad
|10\rangle = \begin{pmatrix}0 \\ 0 \\ 1 \\ 0\end{pmatrix}, \quad
|11\rangle = \begin{pmatrix}0 \\ 0 \\ 0 \\ 1\end{pmatrix}
$$

---

### 🎛️ Hadamard Gate (H)

Creates a superposition from a classical state.

$$
H|0\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle + |1\rangle\right) \\
H|1\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle - |1\rangle\right)
$$

Matrix representation:

$$
H = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
$$

---

🛠️ *To apply a Hadamard gate in Qiskit*:

```python
qc.h(0)  # Applies Hadamard to qubit 0


In [ ]:
#@title ↓Click "run" to see Hadamard Circuit{ display-mode: "form" }
from qiskit import QuantumCircuit
qc=QuantumCircuit(1)
qc.h(0)
qc.draw("mpl")

### 🔁 CNOT (Controlled-NOT) Gate

The **CNOT** gate is a **two-qubit gate**:

- It uses one **control qubit** and one **target qubit**.
- If the **control qubit is in state** $|1\rangle$, the **target qubit flips**.
- If the **control qubit is in state** $|0\rangle$, the **target remains unchanged**.


---

### Gate Behavior (Little-endian notation)

Let qubit **A** be the **control** (rightmost qubit), and qubit **B** be the **target** (leftmost qubit).  
We write the two-qubit state as $CNOT(q_{control},q_{target})|BA\rangle.$

Then:

$$CNOT(A,B)|00\rangle = |00\rangle, \\ CNOT(A,B)|01\rangle = |11\rangle, \\ CNOT(A,B)|10\rangle = |10\rangle, \\ CNOT(A,B)|11\rangle = |01\rangle$$

---

### 🧮 Matrix Representation

The matrix form of the CNOT gate (on two qubits) is:

\[
\text{CNOT} =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0
\end{bmatrix}
\]

---

### Qiskit Usage

In Qiskit, apply a CNOT gate with:

```python
qc.cx(control_qubit, target_qubit)


In [ ]:
#@title ↓Click "run" to see CNot Gate { display-mode: "form" }
qc=QuantumCircuit(2)
qc.cx(0,1)
qc.draw("mpl")

# Theory: Understanding Quantum Teleportation


In this section, we'll walk through the **math** behind quantum teleportation.  
Don’t worry — we'll implement this step-by-step using Qiskit next.

---

### Step 1: Alice and Bob Entangle Their Qubits

Initially, Alice and Bob each have a qubit in the state $|0\rangle$.

Their joint state is:
$|0\rangle_B|0\rangle_A$ = $|00\rangle$


This is called the **little-endian convention**:  
Qubit 0 is on the **right** (used by Qiskit).

---

### Step 2: Apply Gates to Create Entanglement

1. Apply a **Hadamard gate** to Alice’s qubit (A):

$H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$

So their system becomes:

$|0\rangle_B \otimes \frac{1}{\sqrt{2}}(|0\rangle_A + |1\rangle_A)$

2. Now apply a **CNOT gate**:  
Control = Alice's qubit (A)  
Target = Bob's qubit (B)

We calculate:

$CNOT(A, B)\left( |0\rangle_B \cdot \frac{1}{\sqrt{2}}(|0\rangle_A + |1\rangle_A) \right)$

Expanding this:

$\frac{1}{\sqrt{2}} \left( CNOT|0\rangle_B|0\rangle_A + CNOT|0\rangle_B|1\rangle_A \right)$

Using CNOT rules:

- $CNOT|00\rangle = |00\rangle$
- $CNOT|01\rangle = |11\rangle$

So the result is:

$\frac{1}{\sqrt{2}} \left( |0\rangle_B|0\rangle_A + |1\rangle_B|1\rangle_A \right)$

---

### ✅ Result: Bell State

Alice and Bob now share the entangled state:

$\frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$

- It is **not yet determined** whether both qubits are in 0 or 1.
- But we **do know**: they will be the same.

This is called a **Bell state** (or an **EPR pair**).  
It’s the fundamental resource used in **quantum teleportation**.


---

### 🚶‍♀️ Alice and Bob Separate

Now, Alice and Bob **move their entangled qubits** to distant locations.  
This isn’t easy in practice — moving quantum states without introducing noise is tricky.

📌 In this module, you'll simulate this transfer. But just keep in mind that in real-world systems, **moving qubits can introduce errors**.

---

### Introducing Q: The Secret State

Next, a **third qubit**, called **Q**, is introduced.  
It holds the **secret quantum state** that Alice wants to teleport to Bob.

We define Q's state as:

$$
|\psi\rangle_Q = \alpha_0 |0\rangle_Q + \alpha_1 |1\rangle_Q
$$

This state is a general superposition, with:

- $\alpha_0$, $\alpha_1 \in \mathbb{C}$
- $|\alpha_0|^2 + |\alpha_1|^2 = 1$

---

### Current System State

So far, qubit Q is **not entangled** with anything — it just lives next to Alice's qubit (A).

The full 3-qubit system (Bob’s qubit B, Alice’s qubit A, and Q) is:

$$
|\psi\rangle_{ABQ} = \frac{1}{\sqrt{2}}\left( |0\rangle_B|0\rangle_A + |1\rangle_B|1\rangle_A \right)
\left( \alpha_0 |0\rangle_Q + \alpha_1 |1\rangle_Q \right)
$$

We’re about to **move the quantum information** stored in Q **from Alice to Bob**.

📌 No claims yet about **speed** or **security** — just that this is a way to **transmit quantum state data** across space using entanglement + classical communication.


---

### 🔁 Entangling Qubit A with Qubit Q

Because the information **starts on qubit Q**, we assign it the **lowest qubit index**.  
This follows **little-endian notation**, so in tensor products, **Q appears right-most**.

---

### Alice's Operations

Now, Alice applies two gates:

1. A **CNOT** gate with **A as control** and **Q as target**  
2. A **Hadamard** gate on **Q**

---

### 🧮 New Three-Qubit State

Let’s compute the system state after these operations:

$$
\begin{aligned}
H_Q \cdot \text{CNOT}_{A,Q} \cdot |\psi\rangle_{AB} \otimes |\psi\rangle_Q &= H_Q \cdot \text{CNOT}_{A,Q} \cdot \frac{1}{\sqrt{2}} \left( |0\rangle_B |0\rangle_A + |1\rangle_B |1\rangle_A \right) \left( \alpha_0 |0\rangle_Q + \alpha_1 |1\rangle_Q \right) \\
&= H_Q \cdot \text{CNOT}_{A,Q} \cdot \frac{1}{\sqrt{2}} \left[
\alpha_0 |0\rangle_B |0\rangle_A |0\rangle_Q +
\alpha_1 |0\rangle_B |0\rangle_A |1\rangle_Q \right. \\
&\quad \left. + \alpha_0 |1\rangle_B |1\rangle_A |0\rangle_Q +
\alpha_1 |1\rangle_B |1\rangle_A |1\rangle_Q
\right] \\
&= H_Q \cdot \frac{1}{\sqrt{2}} \left[
\alpha_0 |0\rangle_B |0\rangle_A |0\rangle_Q +
\alpha_1 |0\rangle_B |1\rangle_A |1\rangle_Q \right. \\
&\quad \left. + \alpha_0 |1\rangle_B |1\rangle_A |0\rangle_Q +
\alpha_1 |1\rangle_B |0\rangle_A |1\rangle_Q
\right] \\
&= \frac{1}{2} \left[
\alpha_0 |0\rangle_B |0\rangle_A |0\rangle_Q +
\alpha_0 |0\rangle_B |0\rangle_A |1\rangle_Q +
\alpha_1 |0\rangle_B |1\rangle_A |0\rangle_Q -
\alpha_1 |0\rangle_B |1\rangle_A |1\rangle_Q \right. \\
&\quad \left. + \alpha_0 |1\rangle_B |1\rangle_A |0\rangle_Q +
\alpha_0 |1\rangle_B |1\rangle_A |1\rangle_Q +
\alpha_1 |1\rangle_B |0\rangle_A |0\rangle_Q -
\alpha_1 |1\rangle_B |0\rangle_A |1\rangle_Q
\right]
\end{aligned}
$$

---

✅ At this point:
- Qubits **A and Q** are now **entangled**
- Quantum information has been **spread across all three qubits**
- We’re close to **completing the teleportation protocol**


---

### 🔍 Grouping by Measurement Outcomes (A and Q)

Since qubits **A** and **Q** are located together, let’s group the terms by the possible **measurement outcomes** on those two qubits:

\[
\begin{aligned}
|\psi\rangle &= \frac{1}{2} \Big[
(\alpha_0 |0\rangle_B + \alpha_1 |1\rangle_B) |0\rangle_A |0\rangle_Q \\
&\quad + (\alpha_0 |0\rangle_B - \alpha_1 |1\rangle_B) |0\rangle_A |1\rangle_Q \\
&\quad + (\alpha_1 |0\rangle_B + \alpha_0 |1\rangle_B) |1\rangle_A |0\rangle_Q \\
&\quad + (-\alpha_1 |0\rangle_B + \alpha_0 |1\rangle_B) |1\rangle_A |1\rangle_Q
\Big]
\end{aligned}
\]

---

<div class="alert alert-block alert-info">
<b>💡 Check-in Question:</b>  
<p>Given the expression above for the states of all three qubits, what is the probability that a measurement of qubits A and Q yields
$|0\rangle_A |0\rangle_Q$?</p>

<details>
<summary><b>Answer</b></summary>

<p>✅ <b>25%</b>.  

To calculate this, we focus on the amplitude of the term  
\[
\frac{1}{2}(\alpha_0 |0\rangle_B + \alpha_1 |1\rangle_B) |0\rangle_A |0\rangle_Q
\]

The probability is the square of the amplitude norm:

\[
\left| \frac{1}{2} (\alpha_0 |0\rangle_B + \alpha_1 |1\rangle_B) \right|^2 = \frac{1}{4}
\]

</p>
</details>
</div>


---

## Step-by-Step: Alice's Measurement and Classical Communication

Now, Alice performs measurements on **qubits A and Q**. These measurements are **probabilistic**, so she cannot control the outcome.

There are **4 possible outcomes** from measuring qubits A and Q:

- $|0\rangle_A|0\rangle_Q$
- $|0\rangle_A|1\rangle_Q$
- $|1\rangle_A|0\rangle_Q$
- $|1\rangle_A|1\rangle_Q$

Each outcome **collapses the 3-qubit state** and leaves Bob's qubit in a different quantum state. Let's explore what Bob receives depending on Alice's result:

---

### 📊 Teleportation Result Table

| Alice's Outcome | Bob’s State After Collapse | Instruction to Bob | Final State (Recovered) |
|------------------|-----------------------------|----------------------|--------------------------|
| $|0\rangle_A|0\rangle_Q$ | $\alpha_0|0\rangle_B + \alpha_1|1\rangle_B$ | None | $\alpha_0|0\rangle + \alpha_1|1\rangle$ |
| $|0\rangle_A|1\rangle_Q$ | $\alpha_0|0\rangle_B - \alpha_1|1\rangle_B$ | Apply $Z$ | $\alpha_0|0\rangle + \alpha_1|1\rangle$ |
| $|1\rangle_A|0\rangle_Q$ | $\alpha_1|0\rangle_B + \alpha_0|1\rangle_B$ | Apply $X$ | $\alpha_0|0\rangle + \alpha_1|1\rangle$ |
| $|1\rangle_A|1\rangle_Q$ | $-\alpha_1|0\rangle_B + \alpha_0|1\rangle_B$ | Apply $X$ then $Z$ | $\alpha_0|0\rangle + \alpha_1|1\rangle$ |

---

### Summary

- Bob's qubit **always ends up with a scrambled version** of the original state depending on Alice's measurement result.
- Alice must use **classical communication** to tell Bob which correction (if any) to apply.
- Only then can Bob **recover the exact secret state**.

---

### ❗ Why This Can't Transmit Info Faster Than Light

Even if Bob instantly ends up with the correct quantum state, **he doesn’t know that** until Alice tells him via classical communication. So the teleportation **doesn't allow faster-than-light signaling**.

---

### 🧠 Implementation Note

In the real IBM quantum chips, we **can't physically move qubits** to remote locations. Instead:

- We simulate distance using **SWAP gates**
- These move quantum states between physical qubits on the same chip

This lets us model Alice and Bob's separation in teleportation experiments.

---


# Experiment 1: Basic teleportation

---

## IBM's Qiskit Pattern for Quantum Problem Solving

IBM Quantum recommends tackling quantum computing problems using a framework called **Qiskit Patterns**. It breaks down any quantum workflow into 4 key steps:

### 🧩 Qiskit Pattern Steps:

1. **Map your problem** to a quantum circuit
2. **Optimize the circuit** for real quantum hardware
3. **Execute the job** using Runtime Primitives on IBM Quantum systems
4. **Post-process the results** to interpret the output

---

### 4.1 Step 1: Map Your Problem to a Quantum Circuit

In the previous math section, we **mathematically described the teleportation process**. Now, it's time to translate that into a quantum circuit using **Qiskit**.

We will:

- Create a quantum circuit with **3 qubits**:
  - **Qubit 0** → Secret state ($|\psi\rangle$)
  - **Qubit 1** → Alice's entangled qubit
  - **Qubit 2** → Bob's entangled qubit

---

We'll begin by preparing **Alice and Bob's entangled pair**:

- Apply a **Hadamard gate** to qubit 1 (Alice)
- Follow it with a **CNOT gate** from qubit 1 (control) to qubit 2 (target)

This setup will place qubits 1 and 2 into the Bell state:
$$\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$$

Once this entangled pair is ready, we will proceed to prepare the secret quantum state on **qubit 0** and perform teleportation!

---


## Circuit Setup and Entanglement

In [ ]:
# Step 1: Set up the quantum registers and create entanglement between Alice and Bob

from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
? numpy as ?
? ? import pi

# TODO: Define quantum registers for secret, Alice, and Bob
secret = QuantumRegister(?, "Q")
Alice = QuantumRegister(?, "A")
Bob = QuantumRegister(?, "B")

# TODO: Create a classical register for 3 bits
cr = ClassicalRegister(?, 'c')

# TODO: Create the quantum circuit
qc = QuantumCircuit(secret, Alice, Bob, cr)

# TODO: Entangle Alice and Bob’s qubits using H and CNOT
qc.?(?)  # Apply Hadamard to Alice
qc.?(?, ?)  # CNOT with Alice as control, Bob as target

qc.?(?)  # Add barrier for circuit clarity


## Prepare Secret State and Entangle with Alice

In [ ]:
# Step 2: Prepare a secret quantum state and entangle it with Alice's qubit

# TODO: Create random angles for a generic single-qubit state
np.random.seed(42)  # Seed for repeatability
theta = np.random.uniform(?, ?) * np.pi
varphi = np.random.uniform(?, ?) * np.pi

# TODO: Apply a general single-qubit unitary to the secret qubit
qc.u(?, ?, ?, ?)  # Fill in theta, varphi, lambda, qubit

qc.?(?)  # Barrier before next step

# TODO: Entangle the secret qubit with Alice’s using CNOT and H
qc.?(?, ?)  # CNOT: secret → Alice
qc.?(?)     # Hadamard on secret

qc.?(?)  # Barrier to separate measurement


## Measurement and Conditional Logic

In [ ]:
# Step 3: Measure and use classical logic to apply corrections

# TODO: Measure Alice's and secret's qubits into classical bits
qc.measure(?, ?)  # Alice → cr[1]
qc.measure(?, ?)  # Secret → cr[0]

# TODO: Apply conditional gates to Bob based on Alice’s measurements
with qc.if_test((?, ?)):  # If Alice’s qubit is 1
    qc.?(?)  # Apply X to Bob

with qc.if_test((?, ?)):  # If secret's qubit is 1
    qc.?(?)  # Apply Z to Bob

# Visualize the full circuit
qc.draw(output="mpl")


---

That's all we have to do to get **Alice's state teleported to Bob**.

However, recall that when we measure a quantum state  
$\alpha_0|0\rangle + \alpha_1|1\rangle$  
we find either $|0\rangle$ or $|1\rangle$.

So at the end of all this, **Bob definitely has Alice's secret state**,  
but we **can't easily verify** this with a measurement.

---

### 🧠 Verification Trick

In order for a measurement to tell us that we did this correctly, we have to do a trick.

We had an operator labeled **"U" for unitary** which we used to prepare Alice's secret state.

We can apply the **inverse of $U$**, written as $U^\dagger$, at the end of our circuit.

If $U$ mapped Alice’s $|0\rangle$ state into $\alpha_0|0\rangle + \alpha_1|1\rangle$,  
then the **inverse** of $U$ will map Bob’s $\alpha_0|0\rangle + \alpha_1|1\rangle$ back to $|0\rangle$.

---

So this last part **wouldn’t necessarily be done** if the goal were just to **move quantum information**.  
This is **only done for us to check ourselves**.

---


In [ ]:
# Step 4 (verify): Apply the inverse of the original U to Bob’s qubit and measure to validate teleportation

# TODO: Add a barrier for circuit clarity
qc.?(?)  # Barrier

# TODO: Apply the inverse of the secret unitary to Bob's qubit
# Hint: Use .inverse() on qc.u(...) directly, or define U separately
qc.?(?, ?, ?, ?).?  # Apply inverse of U(theta, varphi, 0.0) to Bob

# TODO: Measure Bob’s qubit into the final classical register bit (cr[2])
qc.?(?, ?)  # Measure Bob → cr[2]

# Visualize the final circuit
qc.draw(output="mpl")


# Adaption Activity

## Step 2 – Optimize for Quantum Execution

In [ ]:
# Step 2: Optimize your circuit for a real quantum backend

# TODO: Import and load the Qiskit Runtime Service
from qiskit_ibm_runtime import QiskitRuntimeService
service = ?  # Initialize the service

# TODO: Choose a backend – you may pick the least busy or specify one
backend = service.?(?)  # Select a backend with enough qubits
# backend = service.backend("ibm_brisbane")

print("Backend selected:", backend.name)

# TODO: Transpile the circuit using a preset pass manager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
target = backend.?  # Get the device’s target model
pm = generate_preset_pass_manager(?, optimization_level=?)  # Choose your optimization level
qc_isa = ?.?(?)  # Transpile the circuit


##Step 3 – Run the Circuit

In [ ]:
# Step 3: Execute the quantum teleportation circuit

# TODO: Load the appropriate Qiskit Runtime Sampler
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode = ?)  # Provide the correct backend

# TODO: Submit the job and get the result
job = ?.?(?)  # Run the circuit
res = ?.result()

# TODO: Extract counts from the result
counts = res[?].data.c.get_counts()


# Experiment 2: Teleporting across a processor

For learning and observation, just run these

## 5.1. Step 1: Map Your Problem to a Quantum Circuit (Long-Distance Teleportation)

Arguably, the most interesting part of quantum teleportation is that a **quantum state can be teleported over long distances instantly** (though the classical communication of extra gates is not instant).

As already stated, we can't break qubits off the processor and move them around.  
🔁 **But we can move the *information* from one qubit to another**, until the qubits involved in teleportation are on opposite sides of the processor.

---

Let us **repeat the steps** we took above, but this time we'll:

✅ Create a **larger circuit** with enough qubits to span the processor  
✅ Use **swap gates** to push quantum information from one side of the chip to another

---

> ℹ️ This time, the qubits corresponding to Alice and Bob will change.  
> So we will **not name a single qubit** "A" and another "B".  
> Rather, we will **number the qubits** and use **variables** to represent the current position of the information on qubits belonging to Alice and Bob.

All other steps — except the swap gates — are as described previously.


In [ ]:
# Step 1: Map

# Define registers
qr = QuantumRegister(13, 'q')

qc = QuantumCircuit(qr, cr)

# Define registers
secret = QuantumRegister(1, "Q")
ebitsa = QuantumRegister(6, "A")
ebitsb = QuantumRegister(6, "B")
#q = ClassicalRegister(1, "q meas")
#a = ClassicalRegister(1, "a")
#b = ClassicalRegister(1, "b")
cr = ClassicalRegister(3, 'c')
qc = QuantumCircuit(secret, ebitsa, ebitsb, cr)

# We'll start Alice in the middle of the circuit, then move information outward in both directions.
Alice = 5
Bob = 0
qc.h(ebitsa[Alice])
qc.cx(ebitsa[Alice],ebitsb[Bob])

# Starting with Bob and Alice in the center, we swap their information onto adjacent qubits, until the information is on distant qubits.

for n in range(Alice):
    qc.swap(ebitsb[Bob],ebitsb[Bob+1])
    qc.swap(ebitsa[Alice],ebitsa[Alice-1])
    Alice = Alice-1
    Bob = Bob+1

qc.barrier()

# Create a random state for Alice (qubit zero)
np.random.seed(42) #fixing seed for repeatibility
#theta = np.random.uniform(0.0, 1.0) * np.pi    #from 0 to pi
theta = 0.3
varphi = np.random.uniform(0.0, 2.0) * np.pi    #from 0 to 2*pi


qc.u(theta, varphi, 0.0, secret)

# Entangle Alice's two qubits
qc.cx(secret,ebitsa[Alice])
qc.h(secret)

qc.barrier()

# Make measurements of Alice's qubits and store the results in the classical register.
qc.measure(ebitsa[Alice], cr[1])
qc.measure(secret, cr[0])

# Send instructions to Bob's qubits based on the outcome of Alice's measurements.
with qc.if_test((cr[1], 1)):
    qc.x(ebitsb[Bob])
with qc.if_test((cr[0], 1)):
    qc.z(ebitsb[Bob])

qc.barrier()

# Invert the preparation we did for Carl's qubit so we can check whether we did this correctly.
qc.u(theta, varphi, 0.0, ebitsb[Bob]).inverse()    # inverse of u(theta,varphi,0.0)
qc.measure(ebitsb[Bob], cr[2]) # add measurement gate

qc.draw('mpl')

## 5.2. Step 2: Optimize Your Circuit

You can see in the circuit diagram that the **logical steps are the same**.  
The only difference is that we used **swap gates** to:

- Bring **Alice's qubit state** from qubit 6 ($A_5$) up to qubit 1 ($A_0$), right next to $Q$
- Bring **Bob's initial state** from qubit 7 ($B_0$) down to qubit 12 ($B_5$)

> ⚠️ Note: The state on qubit 12 is **not even related to Q’s secret state** until:
> - Measurements are made on the distant qubits 0 and 1
> - And the **conditional $X$ and $Z$ gates** are applied

---

###  Why We Disable Optimization

Normally, when we use the **pass manager** to transpile and optimize our circuits, it makes sense to set:

```python
optimization_level = 3
```

because we want our circuits to be as efficient as possible.

But in this teleportation experiment:

- The **swap gates** are not logically necessary from a computational perspective  
- They are added **only to demonstrate teleportation over distance**

If we set `optimization_level = 3`, the transpiler will **remove the swap gates**, realizing they are unnecessary, and execute the gates on adjacent qubits instead.

So, for this demonstration, we set:

```python
optimization_level = 0
```

to preserve the swap gates and the physical layout of our circuit.


In [ ]:
# Step 2: Transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pmzero = generate_preset_pass_manager(target=target, optimization_level=0)

qc_isa_zero = pmzero.run(qc)

print(qc_isa_zero.depth())

## 5.3 Step 3: Run the Teleportation Circuit

Before we run the circuit, we can **visualize where the qubits are located** on the actual quantum processor using:

```python
qc_transpiled.draw('mpl')  # Optional
plot_circuit_layout(qc_transpiled, backend)
```

This helps us **see the physical locations** of the qubits on the chip and verify that teleportation is happening over a distance.

---

### Run the Circuit

As before, we recommend executing the circuit on a **real IBM Quantum device**.

But if:

- You're out of free quantum runtime minutes this month
- Or want quicker results

You can use the **simulator** instead. Just uncomment the simulator cell in your notebook:

```python
# job = execute(qc_transpiled, backend_sim, shots=1024)
```

Or continue using the **runtime execution**:

```python
job = backend.run(qc_transpiled, shots=1024)
```

Make sure to wait for job completion and retrieve the results to analyze the teleportation!


In [ ]:
# This required 5 s to run on ibm_torino on 10-28-24
job = sampler.run([qc_isa_zero])
#job = sampler_sim.run([qc_isa_zero])
counts=job.result()[0].data.c.get_counts()

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Step 4: Classical Post-Processing

After executing the quantum teleportation circuit, we now analyze the **measurement results**.

You should observe:

- The measurement probabilities for **Alice's qubits** are fairly **uniform** across the 4 possible outcomes.
- A **strong preference** for measuring **Bob’s qubit in the $|0\rangle$ state**, **after** applying the inverse of the secret preparation circuit.

✅ This high probability of $|0\rangle$ indicates that the **teleportation was successful**, and Bob's qubit likely holds the same quantum information as Alice's original secret qubit (Q).

---

⚠️ However, you might also notice a **slightly increased chance of measuring $|1\rangle$ on Bob’s qubit**.

This is an important insight:

> The more gates (especially **multi-qubit gates** like **SWAP** or **CNOT**) you include, the **more noise and error** accumulates in real quantum hardware.

That's why optimizing quantum circuits — and minimizing the number of operations — is crucial for **real-world quantum applications**.
